In [1]:
!pip install crewai langchain-community langchain-openai python-dotenv duckduckgo-search

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached openai-1.75.0-py3-none-any.whl.metadata (25 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 2.3 MB/s eta 0:00:00a 0:00:01
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [50]:
!pip install --upgrade langchain langchain-community langchain-ollama

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59
  Attempting uninstall: langchain-ollama
    Found existing installation: langchain-ollama 0.3.2
    Uninstalling langchain-ollama-0.3.2:
      Successfully uninstalled langchain-ollama-0.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
app 0.1.0 requires fastapi<0.116.0,>=0.115.12, but you have fastapi 0.115.9 which is incompatible.


In [68]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
# Para este ejemplo, usaremos una herramienta de búsqueda simple de DuckDuckGo
# Crew AI se integra bien con herramientas de Langchain
from langchain_community.tools import DuckDuckGoSearchRun

from litellm import completion

In [69]:
from crewai import LLM
llm=LLM(model="ollama/llama3.2:latest", base_url="http://localhost:11434")


In [78]:
from crewai.tools import BaseTool
from langchain_community.tools import DuckDuckGoSearchRun

class DuckDuckGoSearchTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Realiza búsquedas en la web utilizando DuckDuckGo."

    def _run(self, query: str) -> str:
        search = DuckDuckGoSearchRun()
        return search.run(query)
    
search_tool = DuckDuckGoSearchTool()

In [86]:
researcher_agent = Agent(
    role="Investigador de Destinos de Viaje Senior",
    goal="Descubrir atracciones imperdibles y joyas ocultas en {city} para un viaje de fin de semana.",
    backstory=(
        "Eres un investigador de viajes de renombre con un ojo agudo para los detalles y una pasión por descubrir "
        "experiencias únicas. Tienes una habilidad especial para encontrar información que va más allá de las típicas "
        "guías turísticas. Tu objetivo es proporcionar una lista concisa pero inspiradora de lugares y actividades."
    ),
    tools=[],  # Sin herramientas
    llm=llm,
    verbose=True,
    allow_delegation=False
)

In [87]:
planner_agent = Agent(
    role="Redactor Experto de Itinerarios de Viaje",
    goal="Crear un itinerario de fin de semana atractivo, bien estructurado y factible para {city}, "
         "basado en la investigación proporcionada.",
    backstory=(
        "Eres un talentoso escritor de viajes que se especializa en transformar listas de atracciones en "
        "itinerarios cautivadores y prácticos. Sabes cómo equilibrar actividades, tiempos de viaje y "
        "descansos para crear una experiencia memorable. Tu itinerario debe ser fácil de seguir e inspirador."
    ),
    tools=[],  # Sin herramientas
    llm=llm,
    verbose=True,
    allow_delegation=False
)

print("Agentes definidos.")

Agentes definidos.


In [88]:
# Tarea 1: Investigación del Destino
research_task = Task(
    description=(
        "Investiga a fondo la ciudad de {city}. Identifica las 5 principales atracciones turísticas "
        "y al menos 3 joyas ocultas o experiencias locales únicas. Considera una variedad de intereses "
        "(historia, arte, gastronomía, naturaleza, etc.)."
    ),
    agent=researcher_agent,
    expected_output=(
        "Un informe conciso que liste las 5 atracciones principales y 3 joyas ocultas para {city}, "
        "con una breve descripción (1-2 frases) para cada una. El informe debe ser claro y fácil de entender."
    )
)

# Tarea 2: Creación del Itinerario
# Esta tarea dependerá del resultado de la tarea de investigación.
# Crew AI maneja el paso de contexto entre tareas secuenciales.
planning_task = Task(
    description=(
        "Usando la investigación proporcionada sobre {city} (atracciones principales y joyas ocultas), "
        "crea un itinerario detallado para un viaje de fin de semana (Sábado y Domingo). "
        "El itinerario debe ser atractivo, incluir horarios sugeridos, y ser realista en términos de logística. "
        "Asegúrate de que el tono sea inspirador."
    ),
    agent=planner_agent,
    expected_output=(
        "Un itinerario de fin de semana completo y bien formateado para {city}, presentado día por día (Sábado, Domingo). "
        "Debe incluir nombres de lugares, breves descripciones de actividades y sugerencias de horarios. "
        "El resultado final debe ser un único bloque de texto Markdown."
    ),
    # context=[research_task] # Opcionalmente, puedes definir explícitamente el contexto de tareas anteriores.
    # CrewAI a menudo lo infiere bien en procesos secuenciales.
)

print("Tareas definidas.")

Tareas definidas.


In [89]:
trip_crew = Crew(
    agents=[researcher_agent, planner_agent],
    tasks=[research_task, planning_task],
    process=Process.sequential, # Las tareas se ejecutarán en el orden en que se listan
    verbose=True # Nivel de verbosidad: 0 (silencioso), 1 (básico), 2 (detallado)
)

print("Tripulación creada.")

Tripulación creada.


In [90]:
city_to_plan = "París"

print(f"\n--- Iniciando la Tripulación para planificar un viaje a: {city_to_plan} ---")


--- Iniciando la Tripulación para planificar un viaje a: París ---


In [91]:
# Ejecutar la tripulación con las entradas necesarias
try:
    trip_result = trip_crew.kickoff(inputs={'city': city_to_plan})

    print("\n\n--- Resultado Final de la Tripulación ---")
    print(trip_result)

except Exception as e:
    print(f"Error durante la ejecución de la tripulación: {e}")



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7773db73-f544-446f-a514-e151fddfb953                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5db2ed3d-080c-4d6f-aeb7-5db445c90cf0
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5db2ed3d-080c-4d6f-aeb7-5db445c90cf0
       Status: Executing Task...
    └── 🤖 Agent: Investigador de Destinos de Viaje Senior
            Status: In Progress

# Agent: Investigador de Destinos de Viaje Senior
## Task: Investiga a fondo la ciudad de París. Identifica las 5 principales atracciones turísticas y al menos 3 joyas ocultas o experiencias locales únicas. Considera una variedad de intereses (historia, arte, gastronomía, naturaleza, etc.).


# Agent: Investigador de Destinos de Viaje Senior
## Final Answer: 
### Atracciones Turísticas Principales:

1.  **La Torre Eiffel**: Es uno de los monumentos más icónicos y emblemáticos de París, con una historia rica que abarca desde su construcción hasta la actualidad.

2.  **El Louvre**: Ubicado en el corazón del Museo Nacional del Lujo, este museo cuenta con una colección impresionante de arte y objetos históricos que atrapan a todos los visitantes.

3.  **La Catedral de Notre-Dame**: Este imponente edificio gótico es uno de los símbolos más reconocibles de la ciudad, cuyas paredes han testigo de décadas de historia y tradiciones.

4.  **El Jardín de Luxemburgo**: Este hermoso jardín es un lu

🚀 Crew: crew
└── 📋 Task: 5db2ed3d-080c-4d6f-aeb7-5db445c90cf0
       Status: Executing Task...
    └── 🤖 Agent: Investigador de Destinos de Viaje Senior
            Status: ✅ Completed

...

🚀 Crew: crew
├── 📋 Task: 5db2ed3d-080c-4d6f-aeb7-5db445c90cf0
│      Assigned to: Investigador de Destinos de Viaje Senior
│      Status: ✅ Completed
│   └── 🤖 Agent: Investigador de Destinos de Viaje Senior
│           Status: ✅ Completed
└── 📋 Task: 880196f7-2aff-4cdf-889d-97400594b32e
       Status: Executing Task...
    └── 🤖 Agent: Redactor Experto de Itinerarios de Viaje
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 5db2ed3d-080c-4d6f-aeb7-5db445c90cf0
│      Assigned to: Investigador de Destinos de Viaje Senior
│      Status: ✅ Completed
│   └── 🤖 Agent: Investigador de Destinos de Viaje Senior
│           Status: ✅ Completed
└── 📋 Task: 880196f7-2aff-4cdf-889d-97400594b32e
       Assigned to: Redactor Experto de Itinerarios de Viaje
       Status: ✅ Completed
    └── 🤖 Agent: Redactor Experto de Itinerarios de Viaje
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 880196f7-2aff-4cdf-889d-97400594b32e                                                                     │
│  Agent: Redactor Experto de Itinerarios de Viaje                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7773db73-f544-446f-a514-e151fddfb953                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



--- Resultado Final de la Tripulación ---
#### Sábado

**8:00 am - Desayuno**

*   Comienza el fin de semana con un desayuno tradicional francés en un café local, como Café de Flore o Les Deux Magots, donde podrás disfrutar de pan tostado y mantequilla casera.

**9:30 am - La Torre Eiffel**

*   Dirígete a la Torre Eiffel para explorar sus diferentes niveles y disfrutar de las vistas panorámicas impresionantes de la ciudad. Recuerda comprar tus entradas con anticipación para evitar las longas filas.

**12:30 pm - El Louvre**

*   Después de una visita emocionante a la Torre Eiffel, dirígete al Museo del Louvre para explorar su impresionante colección de arte y objetos históricos. No te pierdas la famosa Mona Lisa.

**3:30 pm - El Jardín de Luxemburgo**

*   Después de un día intenso de visitas turísticas, relájate en el hermoso Jardín de Luxemburgo. Disfruta del paisaje natural y de las vistas impresionantes que te ofrecen los edificios históricos circundantes.

**7:00 pm - Cena**

*

In [92]:
# Otro ejemplo
city_to_plan_2 = "Kioto"
print(f"\n--- Iniciando la Tripulación para planificar un viaje a: {city_to_plan_2} ---")
try:
    trip_result_2 = trip_crew.kickoff(inputs={'city': city_to_plan_2})
    print("\n\n--- Resultado Final de la Tripulación (Kioto) ---")
    print(trip_result_2)
except Exception as e:
    print(f"Error durante la ejecución de la tripulación para Kioto: {e}")


--- Iniciando la Tripulación para planificar un viaje a: Kioto ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7773db73-f544-446f-a514-e151fddfb953                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5db2ed3d-080c-4d6f-aeb7-5db445c90cf0
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5db2ed3d-080c-4d6f-aeb7-5db445c90cf0
       Status: Executing Task...
    └── 🤖 Agent: Investigador de Destinos de Viaje Senior
            Status: In Progress

# Agent: Investigador de Destinos de Viaje Senior
## Task: Investiga a fondo la ciudad de Kioto. Identifica las 5 principales atracciones turísticas y al menos 3 joyas ocultas o experiencias locales únicas. Considera una variedad de intereses (historia, arte, gastronomía, naturaleza, etc.).


# Agent: Investigador de Destinos de Viaje Senior
## Final Answer: 
### Investigación Completa de Kioto

### 5 Atracciones Principales Turísticas

1.  **Templo Kinkaku-ji (El Templo del Golden Peso)**: Uno de los templos más famosos de Kioto, famoso por su hermoso jardín y el espléndido Edificio Temprano de Oro.
2.  **Fushimi Inari Taisha**: Un santuario Shinto conocido por sus miles de puertas vermeltas que forman un paseo al pie de la montaña. Se trata de un lugar sagrado y espiritual para los japoneses, donde se encuentra la diosa de la fertilidad y el crecimiento.
3.  **Castillo Nijo**: Un palacio japonés famoso por sus "joyas del castillo", que son las pequeñas cámaras que hacen clic cuando a

...

🚀 Crew: crew
├── 📋 Task: 5db2ed3d-080c-4d6f-aeb7-5db445c90cf0
│      Assigned to: Investigador de Destinos de Viaje Senior
│      Status: ✅ Completed
│   └── 🤖 Agent: Investigador de Destinos de Viaje Senior
│           Status: ✅ Completed
└── 📋 Task: 880196f7-2aff-4cdf-889d-97400594b32e
       Status: Executing Task...
    └── 🤖 Agent: Redactor Experto de Itinerarios de Viaje
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 5db2ed3d-080c-4d6f-aeb7-5db445c90cf0
│      Assigned to: Investigador de Destinos de Viaje Senior
│      Status: ✅ Completed
│   └── 🤖 Agent: Investigador de Destinos de Viaje Senior
│           Status: ✅ Completed
└── 📋 Task: 880196f7-2aff-4cdf-889d-97400594b32e
       Assigned to: Redactor Experto de Itinerarios de Viaje
       Status: ✅ Completed
    └── 🤖 Agent: Redactor Experto de Itinerarios de Viaje
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 880196f7-2aff-4cdf-889d-97400594b32e                                                                     │
│  Agent: Redactor Experto de Itinerarios de Viaje                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7773db73-f544-446f-a514-e151fddfb953                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



--- Resultado Final de la Tripulación (Kioto) ---
### **Sábado**

**Mañana (9:00 am - 12:30 pm)**

*   **8:00 am:** Comienza el día con una taza de té y un pastel en un café local en el distrito Gion.
*   **9:30 am:** Visita el **Templo Kinkaku-ji (El Templo del Golden Peso)**. Es uno de los templos más famosos de Kioto, famoso por su hermoso jardín y el espléndido Edificio Temprano de Oro.
*   **11:30 am:** Camina al **Fushimi Inari Taisha** para visitar las miles de puertas vermeltas que forman un paseo al pie de la montaña.

**Tarde (1:00 pm - 5:30 pm)**

*   **12:30 pm:** Desayuna en un restaurante local y disfruta de algunos platos de comida japonesa.
*   **1:30 pm:** Visita el **Castillo Nijo**, famoso por sus "joyas del castillo", que son las pequeñas cámaras que hacen clic cuando alguien pisa en ellas, indicando que alguien ha entrado o salido. El castillo también cuenta con hermosos jardines y un palacio de arte.
*   **4:00 pm:** Visita el **Arashiyama Bamboo Forest** para d